In [ ]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from scipy.stats import wilcoxon
import pandas as pd
from drn_interactions.config import ExperimentInfo, Config
from drn_interactions.config import Config
from drn_interactions.decoding.loaders import FSDecodeDataLoader, FSFastDecodeDataLoader
from drn_interactions.decoding.preprocessors import DecodePreprocessor
from drn_interactions.decoding.runners import DecodeRunner
from drn_interactions.decoding.decoders import Decoder
from drn_interactions.decoding.shuffle import shuffle_X

In [ ]:
dd = Config.derived_data_dir / "decoding"
sessions = ExperimentInfo.foot_shock_sessions_10min
session = sessions[0]
loader = FSDecodeDataLoader(session_name=session)
preprocessor = DecodePreprocessor(
    thresh_empty=2,
)

estimator = LGBMClassifier(n_estimators=50, max_depth=8, num_leaves=30)
cv = KFold(n_splits=5, shuffle=True)
decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
runner = DecodeRunner(
    loader=loader,
    preprocessor=preprocessor,
    decoder=decoder,
    nboot=1,
)

In [ ]:
pop, unit, unit_shuff = runner.run_multiple(sessions)
limit = runner.run_multiple_limit(sessions=sessions, n_min=1, n_max=31)
dropout = runner.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))

In [ ]:
pop.to_csv(dd / "fs_slow - pop.csv", index=False)
unit.to_csv(dd / "fs_slow - unit.csv", index=False)
limit.to_csv(dd / "fs_slow - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - dropout.csv", index=False)

In [ ]:
W, p = pd.merge(
    unit_shuff.rename(columns={"F1 Score": "shuff_score"}),
    unit.rename(columns={"F1 Score": "real_score"}),
).pipe(lambda x: wilcoxon(x.shuff_score, x.real_score))
print(W, p)
print(unit_shuff["F1 Score"].median())
print(unit["F1 Score"].median())